In [ ]:
"""
Tree Detection Pipeline - Dependency Installer for Google Colab

Run this ONCE at the start of your Colab session.
After installation, you can reload/modify the main script without reinstalling.

Usage:
%run colab_install_deps.py
"""

print("Installing dependencies for Tree Detection Pipeline...")
print("This will take ~5-10 minutes (ONE TIME only)")

import subprocess
import sys

def install_packages():
    """Install only YOLO + SegFormer requirements"""
    packages = [
        "torch torchvision",
        "transformers>=4.35.0",
        "huggingface_hub",
        "ultralytics>=8.3.184",
        "opencv-python>=4.8.0",
        "tqdm>=4.65.0",
        "PyYAML>=6.0",
        "Pillow>=10.0.0",
        "rasterio",
        "shapely>=2.0.0",
        "pyproj>=3.4.0",
        "requests>=2.25.0",
        "mercantile>=1.2.0",
        "safetensors",
    ]

    for i, package in enumerate(packages, 1):
        print(f"\n[{i}/{len(packages)}] Installing {package.split()[0]}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + package.split())

# Install system dependencies
print("\nInstalling system dependencies...")
subprocess.run(["apt-get", "update", "-qq"], check=True)
subprocess.run(["apt-get", "install", "-y", "-qq", "libgdal-dev", "gdal-bin"], check=True)

# Install Python packages
print("\nInstalling Python packages...")
install_packages()

print("\nAll dependencies installed successfully!")
print("Now run the main script: %run tree_detection_colab_main.py")

Tree Detection Pipeline - YOLO Only (Multi-City Parallel Version)

This is a LIGHTWEIGHT script for tree detection using YOLO with:
- Multi-city support (array of bounding boxes)
- Parallel download and inference
- Resume capability per city

Run this AFTER installing dependencies with colab_install_deps.py.

Usage:
1. First run: Execute colab_install_deps.py (5-10 minutes, once per session)
2. Then run this script (fast, can reload for parameter changes)
3. Edit CONFIGURATION section below as needed
4. Re-run anytime to test new parameters

This notebook uses MapTiler APIs

In [ ]:
# ============================================================================
# PART 1: IMPORT ALL REQUIRED LIBRARIES
# ============================================================================

import os
import json
import logging
import glob
import gc
import time
import threading
from queue import Queue
from typing import Dict, Tuple, List
from dataclasses import dataclass

import torch
import numpy as np
from PIL import Image
import io
import requests
import mercantile

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def setup_city_logger(city_name: str, output_folder: str):
    """Setup a separate logger for each city"""
    city_logger = logging.getLogger(f"city_{city_name}")
    city_logger.setLevel(logging.INFO)

    # Remove existing handlers
    city_logger.handlers = []

    # File handler
    if LOG_TO_FILE:
        log_file = os.path.join(output_folder, f"{city_name}_processing.log")
        file_handler = logging.FileHandler(log_file, mode='a')
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        city_logger.addHandler(file_handler)

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
    city_logger.addHandler(console_handler)

    return city_logger

# ============================================================================
# PART 2: CONFIGURATION
# ============================================================================

print("="*70)
print("CONFIGURATION")
print("="*70)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ============================================================================
# CONFIGURATION - EDIT THESE PARAMETERS
# ============================================================================

# ==== AREAS OF INTEREST (Multi-City Support) ====
# Each entry: {"name": "city_name", "bbox": (west, south, east, north)}
# Use http://bboxfinder.com to get coordinates

CITIES = [
    {"name": "YourCity", "bbox": (0.0, 0.0, 0.0, 0.0)},
]

# ==== YOLO MODEL SETTINGS ====
YOLO_MODEL_PATH = "/content/drive/MyDrive/your_model.pt"
YOLO_CONFIDENCE = 0.5

# ==== MAPTILER API ====
MAPTILER_API_KEY = "YOUR_API_KEY_HERE"  # Get free key at https://www.maptiler.com/cloud/

# ==== PROCESSING SETTINGS ====
ZOOM_LEVEL = 18            # Zoom level (10-18). None for auto-calculation
MAX_TILES = None           # Maximum tiles to process PER CITY. None = process ALL tiles
TILE_SIZE = 512            # Tile size in pixels
DOWNLOAD_THREADS = 8       # Number of parallel download threads
INFERENCE_BATCH_SIZE = 16  # Number of tiles to process together in YOLO (GPU optimization)
SAVE_IMAGES = False        # Save annotated images (takes more space)
LOG_TO_FILE = True         # Save detailed logs to file

# ==== OUTPUT FOLDER ====
OUTPUT_BASE_FOLDER = "/content/drive/MyDrive/detections"

# ============================================================================
# END CONFIGURATION
# ============================================================================

print(f"\nYOLO Model: {YOLO_MODEL_PATH}")
print(f"YOLO Confidence: {YOLO_CONFIDENCE}")
print(f"Cities to process: {len(CITIES)}")
for city in CITIES:
    print(f"  - {city['name']}: {city['bbox']}")
print(f"Download threads: {DOWNLOAD_THREADS}")
print(f"Max Tiles per city: {MAX_TILES}")
print(f"Output base: {OUTPUT_BASE_FOLDER}\n")

# ============================================================================
# PART 3: DATA STRUCTURES
# ============================================================================

@dataclass
class TileRequest:
    """Represents a tile download request"""
    city_name: str
    x: int
    y: int
    zoom: int
    tile_size: int

@dataclass
class TileResult:
    """Represents a downloaded tile ready for processing"""
    city_name: str
    data: Dict
    x: int
    y: int
    zoom: int
    bbox: Tuple

# ============================================================================
# PART 4: UTILITY FUNCTIONS
# ============================================================================

def get_device():
    """Detect and return the best available device"""
    if torch.cuda.is_available():
        device = "cuda"
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        device = "cpu"
        print("Using CPU (will be slower)")
    return device

device = get_device()

# ============================================================================
# PART 5: MAPTILER API CLIENT
# ============================================================================

class MapTilerAPI:
    """MapTiler API integration with thread-safety"""

    def __init__(self, api_key: str, max_retries: int = 3):
        self.api_key = api_key
        self.base_url = "https://api.maptiler.com/maps"
        self.max_retries = max_retries
        # Thread-local session
        self.thread_local = threading.local()

    def _get_session(self):
        """Get thread-local session"""
        if not hasattr(self.thread_local, "session"):
            self.thread_local.session = requests.Session()
        return self.thread_local.session

    def fetch_single_tile(self, x: int, y: int, zoom: int, tile_size: int = 512, format: str = "jpg"):
        """Fetch a single tile and return as georeferenced data with retry logic"""
        url = f"{self.base_url}/satellite/{zoom}/{x}/{y}.{format}"
        params = {"key": self.api_key}
        session = self._get_session()

        # Retry logic with exponential backoff
        for attempt in range(self.max_retries):
            try:
                response = session.get(url, params=params, timeout=30)

                # If status is 500, retry with exponential backoff
                if response.status_code == 500:
                    wait_time = (2 ** attempt)  # 1s, 2s, 4s
                    logger.warning(f"MapTiler returned 500 for tile {x}/{y}/{zoom}. Attempt {attempt + 1}/{self.max_retries}. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                    continue

                response.raise_for_status()

                # Convert to georeferenced array
                image = Image.open(io.BytesIO(response.content))
                if image.mode != 'RGB':
                    image = image.convert('RGB')
                img_array = np.array(image)

                # Get bounds
                n = 2.0 ** zoom
                west = x / n * 360.0 - 180.0
                east = (x + 1) / n * 360.0 - 180.0
                import math
                lat_rad_top = math.atan(math.sinh(math.pi * (1 - 2 * y / n)))
                lat_rad_bottom = math.atan(math.sinh(math.pi * (1 - 2 * (y + 1) / n)))
                north = math.degrees(lat_rad_top)
                south = math.degrees(lat_rad_bottom)

                return {
                    "array": img_array,
                    "bounds": (west, south, east, north),
                    "width": image.width,
                    "height": image.height
                }
            except requests.exceptions.RequestException as e:
                if attempt < self.max_retries - 1:
                    wait_time = (2 ** attempt)
                    logger.warning(f"Request error for tile {x}/{y}/{zoom}: {e}. Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                    continue
                else:
                    logger.error(f"Failed to fetch tile {x}/{y}/{zoom} after {self.max_retries} attempts: {e}")
                    return None
            except Exception as e:
                logger.error(f"Unexpected error fetching tile {x}/{y}/{zoom}: {e}")
                return None

        # If all retries failed
        logger.error(f"Failed to fetch tile {x}/{y}/{zoom} after {self.max_retries} attempts")
        return None

# ============================================================================
# PART 6: PARALLEL DOWNLOAD WORKER
# ============================================================================

class TileDownloader:
    """Manages parallel tile downloads"""

    def __init__(self, api_key: str, num_threads: int = 4):
        self.maptiler = MapTilerAPI(api_key)
        self.num_threads = num_threads
        self.download_queue = Queue()
        self.result_queue = Queue()
        self.threads = []
        self.stop_event = threading.Event()

    def _download_worker(self):
        """Worker thread for downloading tiles"""
        while not self.stop_event.is_set():
            try:
                # Get tile request with timeout to allow checking stop_event
                tile_request = self.download_queue.get(timeout=1)

                if tile_request is None:  # Poison pill
                    break

                # Download tile
                tile_data = self.maptiler.fetch_single_tile(
                    tile_request.x,
                    tile_request.y,
                    tile_request.zoom,
                    tile_request.tile_size
                )

                if tile_data:
                    # Get bbox for this tile
                    n = 2.0 ** tile_request.zoom
                    west = tile_request.x / n * 360.0 - 180.0
                    east = (tile_request.x + 1) / n * 360.0 - 180.0
                    import math
                    lat_rad_top = math.atan(math.sinh(math.pi * (1 - 2 * tile_request.y / n)))
                    lat_rad_bottom = math.atan(math.sinh(math.pi * (1 - 2 * (tile_request.y + 1) / n)))
                    north = math.degrees(lat_rad_top)
                    south = math.degrees(lat_rad_bottom)

                    tile_result = TileResult(
                        city_name=tile_request.city_name,
                        data=tile_data,
                        x=tile_request.x,
                        y=tile_request.y,
                        zoom=tile_request.zoom,
                        bbox=(north, south, east, west)
                    )
                    self.result_queue.put(tile_result)
                else:
                    # Put None to indicate failed download
                    self.result_queue.put(None)

                self.download_queue.task_done()
            except:
                continue

    def start(self):
        """Start download threads"""
        self.stop_event.clear()
        for _ in range(self.num_threads):
            thread = threading.Thread(target=self._download_worker, daemon=True)
            thread.start()
            self.threads.append(thread)

    def add_tile(self, tile_request: TileRequest):
        """Add a tile to download queue"""
        self.download_queue.put(tile_request)

    def stop(self):
        """Stop all download threads"""
        # Send poison pills
        for _ in range(self.num_threads):
            self.download_queue.put(None)

        # Wait for threads to finish
        for thread in self.threads:
            thread.join()

        self.threads = []

# ============================================================================
# PART 7: YOLO INFERENCE FUNCTION (WITH BATCH SUPPORT)
# ============================================================================

def run_yolo_inference_single(image_array: np.ndarray, model, conf: float, device: str):
    """Run YOLO inference on a single image"""
    results_raw = model(image_array, conf=conf, device=device, verbose=False)

    detections = []
    for result in results_raw:
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            
            for i, (box, confidence) in enumerate(zip(boxes, confidences)):
                x1, y1, x2, y2 = box.astype(int)
                detections.append({
                    "id": i,
                    "confidence": float(confidence),
                    "bbox": {"x1": int(x1), "y1": int(y1), "x2": int(x2), "y2": int(y2)}
                })

    return {
        "trees_detected": len(detections),
        "detections": detections,
        "image_size": {"width": image_array.shape[1], "height": image_array.shape[0]}
    }

def run_yolo_inference_batch(image_arrays: List[np.ndarray], model, conf: float, device: str):
    """Run YOLO inference on a batch of images for better GPU utilization"""
    if len(image_arrays) == 0:
        return []

    # Run batch inference
    results_raw = model(image_arrays, conf=conf, device=device, verbose=False)

    batch_results = []
    for result in results_raw:
        detections = []
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()

            for i, (box, confidence) in enumerate(zip(boxes, confidences)):
                x1, y1, x2, y2 = box.astype(int)
                detections.append({
                    "id": i,
                    "confidence": float(confidence),
                    "bbox": {"x1": int(x1), "y1": int(y1), "x2": int(x2), "y2": int(y2)}
                })

        batch_results.append({
            "trees_detected": len(detections),
            "detections": detections,
            "image_size": {"width": result.orig_shape[1], "height": result.orig_shape[0]}
        })

    return batch_results

# ============================================================================
# PART 8: PROCESSING FUNCTIONS
# ============================================================================

def process_tile(tile_result: TileResult, model, yolo_conf: float, device: str):
    """Process a single tile with YOLO detection using pre-loaded model"""
    try:
        image_array = tile_result.data["array"]
        bounds = tile_result.data["bounds"]
        tile_id = f"tile_{tile_result.zoom}_{tile_result.x}_{tile_result.y}"

        # Run YOLO inference with pre-loaded model
        yolo_results = run_yolo_inference_single(image_array, model, yolo_conf, device)

        # Create result
        results = {
            "image_id": tile_id,
            "image_bounds": bounds,
            "image_size": yolo_results["image_size"],
            "tile_x": tile_result.x,
            "tile_y": tile_result.y,
            "tile_zoom": tile_result.zoom,
            "tile_bbox": tile_result.bbox,
            "trees_detected": yolo_results["trees_detected"],
            "detections": yolo_results["detections"],
            "confidence": yolo_conf,
            "city": tile_result.city_name
        }

        return results
    except Exception as e:
        logger.error(f"Error processing tile: {e}")
        return {"error": str(e), "city": tile_result.city_name}

def process_tile_batch(tile_results: List[TileResult], model, yolo_conf: float, device: str):
    """Process a batch of tiles with YOLO for better GPU utilization"""
    if len(tile_results) == 0:
        return []

    try:
        # Prepare batch of images
        image_arrays = [tr.data["array"] for tr in tile_results]

        # Run batch inference
        batch_yolo_results = run_yolo_inference_batch(image_arrays, model, yolo_conf, device)

        # Create results for each tile
        results = []
        for tile_result, yolo_result in zip(tile_results, batch_yolo_results):
            bounds = tile_result.data["bounds"]
            tile_id = f"tile_{tile_result.zoom}_{tile_result.x}_{tile_result.y}"

            result = {
                "image_id": tile_id,
                "image_bounds": bounds,
                "image_size": yolo_result["image_size"],
                "tile_x": tile_result.x,
                "tile_y": tile_result.y,
                "tile_zoom": tile_result.zoom,
                "tile_bbox": tile_result.bbox,
                "trees_detected": yolo_result["trees_detected"],
                "detections": yolo_result["detections"],
                "confidence": yolo_conf,
                "city": tile_result.city_name
            }
            results.append(result)

        return results

    except Exception as e:
        logger.error(f"Error processing batch: {e}")
        # Fall back to single processing
        return [process_tile(tr, model, yolo_conf, device) for tr in tile_results]

def get_last_processed_tile(output_folder):
    """Find the last successfully processed tile to enable resume"""
    json_folder = os.path.join(output_folder, "json")
    if not os.path.exists(json_folder):
        return None

    # Find all result files
    result_files = glob.glob(os.path.join(json_folder, "tile_*_results.json"))
    if not result_files:
        return None

    # Parse tile coordinates from filenames
    tiles_processed = []
    for filepath in result_files:
        try:
            filename = os.path.basename(filepath)
            # Format: tile_zoom_x_y_results.json
            parts = filename.replace("_results.json", "").split("_")
            if len(parts) >= 4 and parts[0] == "tile":
                zoom = int(parts[1])
                x = int(parts[2])
                y = int(parts[3])
                tiles_processed.append((zoom, x, y))
        except (ValueError, IndexError) as e:
            logger.warning(f"Could not parse tile from filename {filename}: {e}")
            continue

    if not tiles_processed:
        return None

    logger.info(f"Found {len(tiles_processed)} previously processed tiles")
    return set(tiles_processed)

# ============================================================================
# PART 9: MAIN PIPELINE (PARALLEL VERSION)
# ============================================================================

def run_pipeline_parallel(city_config: Dict, yolo_model, yolo_conf: float,
                         downloader: TileDownloader, device: str,
                         max_tiles: int, zoom: int):
    """Run the complete detection pipeline for a single city with parallel processing and batch inference"""

    city_name = city_config["name"]
    bbox = city_config["bbox"]

    # Setup output folder for this city
    output_folder = os.path.join(OUTPUT_BASE_FOLDER, f"{city_name}_zoom{zoom}")
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(os.path.join(output_folder, "json"), exist_ok=True)

    # Setup city-specific logger
    city_logger = setup_city_logger(city_name, output_folder)

    city_logger.info(f"{'='*70}")
    city_logger.info(f"PROCESSING CITY: {city_name.upper()}")
    city_logger.info(f"{'='*70}")
    city_logger.info(f"Bounding box: {bbox}")
    city_logger.info(f"Zoom level: {zoom}")
    city_logger.info(f"Output folder: {output_folder}")

    # Check for previously processed tiles
    processed_tiles = get_last_processed_tile(output_folder)
    resume_mode = processed_tiles is not None and len(processed_tiles) > 0

    # Get tile coordinates at specified zoom
    tiles_coords = list(mercantile.tiles(*bbox, zooms=zoom))
    total_tiles = len(tiles_coords)

    city_logger.info(f"Total tiles at zoom {zoom}: {total_tiles}")

    # Filter out already processed tiles if resuming
    if resume_mode:
        original_count = len(tiles_coords)
        tiles_coords = [t for t in tiles_coords if (t.z, t.x, t.y) not in processed_tiles]
        skipped_count = original_count - len(tiles_coords)
        city_logger.info(f"RESUME MODE: Found {len(processed_tiles)} previously processed tiles")
        city_logger.info(f"Skipping {skipped_count} already completed tiles")
        city_logger.info(f"Remaining to process: {len(tiles_coords)} tiles")

    if max_tiles is not None and len(tiles_coords) > max_tiles:
        tiles_coords = tiles_coords[:max_tiles]
        city_logger.info(f"Limiting to {max_tiles} tiles (out of {total_tiles} total)")
    elif not resume_mode:
        city_logger.info(f"Processing ALL {len(tiles_coords)} tiles (no limit)")

    if len(tiles_coords) == 0:
        city_logger.info(f"All tiles already processed for {city_name}! Nothing to do.")
        # Load existing results for stats
        return load_existing_results(output_folder), zoom

    city_logger.info(f"Starting parallel processing of {len(tiles_coords)} tiles...")
    city_logger.info(f"Download threads: {DOWNLOAD_THREADS}")
    city_logger.info(f"Inference batch size: {INFERENCE_BATCH_SIZE}")
    city_logger.info(f"Processing with YOLO on {device}")

    # Queue all tiles for download
    for tile in tiles_coords:
        tile_request = TileRequest(
            city_name=city_name,
            x=tile.x,
            y=tile.y,
            zoom=tile.z,
            tile_size=TILE_SIZE
        )
        downloader.add_tile(tile_request)

    # Process tiles in batches as they come in
    all_results = []
    json_folder = os.path.join(output_folder, "json")
    tiles_to_process = len(tiles_coords)
    processed_count = 0
    failed_count = 0

    # Batch collection
    tile_batch = []
    start_time = time.time()
    last_progress_time = start_time

    while processed_count + failed_count < tiles_to_process:
        try:
            # Collect tiles for batch (with timeout to prevent infinite wait)
            timeout = 2.0 if len(tile_batch) > 0 else 60.0

            try:
                tile_result = downloader.result_queue.get(timeout=timeout)

                if tile_result is None:
                    # Failed download
                    failed_count += 1
                    city_logger.warning(f"Download failed | Progress: {processed_count}/{tiles_to_process} | Failed: {failed_count}")
                    continue

                tile_batch.append(tile_result)

            except:
                # Timeout - process what we have if batch is not empty
                if len(tile_batch) == 0:
                    continue

            # Process batch when full or timeout occurred
            if len(tile_batch) >= INFERENCE_BATCH_SIZE or (len(tile_batch) > 0 and processed_count + failed_count + len(tile_batch) >= tiles_to_process):
                # Process batch with YOLO
                batch_results = process_tile_batch(tile_batch, yolo_model, yolo_conf, device)

                # Save results
                for result in batch_results:
                    all_results.append(result)

                    # Save JSON immediately
                    tile_id = result.get("image_id", "unknown")
                    json_path = os.path.join(json_folder, f"{tile_id}_results.json")
                    with open(json_path, 'w') as f:
                        json.dump(result, f, indent=2)

                    processed_count += 1

                # Progress update
                current_time = time.time()
                if current_time - last_progress_time >= 5.0:  # Update every 5 seconds
                    elapsed = current_time - start_time
                    rate = processed_count / elapsed if elapsed > 0 else 0
                    remaining = tiles_to_process - processed_count - failed_count
                    eta = remaining / rate if rate > 0 else 0

                    total_trees = sum(r.get("trees_detected", 0) for r in all_results)

                    city_logger.info(
                        f"Progress: {processed_count}/{tiles_to_process} | "
                        f"Trees: {total_trees} | "
                        f"Rate: {rate:.1f} tiles/s | "
                        f"ETA: {eta/60:.1f}m | "
                        f"Failed: {failed_count}"
                    )
                    last_progress_time = current_time

                # Clear batch
                tile_batch = []

                # Periodic memory cleanup
                if processed_count % 100 == 0:
                    gc.collect()
                    if device == "cuda":
                        torch.cuda.empty_cache()

        except Exception as e:
            city_logger.error(f"Error in processing loop: {e}")
            failed_count += 1
            tile_batch = []  # Clear batch on error
            continue

    elapsed_total = time.time() - start_time
    city_logger.info(f"{city_name} processing complete!")
    city_logger.info(f"Processed: {processed_count} tiles in {elapsed_total/60:.1f} minutes")
    city_logger.info(f"Average rate: {processed_count/elapsed_total:.2f} tiles/second")
    city_logger.info(f"Failed: {failed_count}")

    return all_results, zoom

def load_existing_results(output_folder):
    """Load existing results from JSON files"""
    json_folder = os.path.join(output_folder, "json")
    all_result_files = glob.glob(os.path.join(json_folder, "tile_*_results.json"))
    results = []
    for filepath in all_result_files:
        with open(filepath, 'r') as f:
            results.append(json.load(f))
    return results

# ============================================================================
# PART 10: GEOJSON CREATION
# ============================================================================

def create_geojson(results, output_folder, city_name):
    """Create GeoJSON from YOLO detection results"""
    yolo_path = os.path.join(output_folder, f'{city_name}_yolo_detections.geojson')

    def pixel_to_latlon(pixel_x, pixel_y, bounds, size):
        west, south, east, north = bounds
        width, height = size['width'], size['height']
        lon = west + (pixel_x / width) * (east - west)
        lat = north - (pixel_y / height) * (north - south)
        return lat, lon

    # Create YOLO GeoJSON
    with open(yolo_path, 'w') as f:
        f.write('{\n')
        f.write('  "type": "FeatureCollection",\n')
        f.write('  "features": [\n')

        first = True
        for result in results:
            if 'error' in result:
                continue

            bounds = result.get('image_bounds')
            size = result.get('image_size')

            if not bounds or not size:
                continue

            # Process YOLO detections
            if 'detections' in result:
                for det in result['detections']:
                    bbox_coords = det['bbox']
                    center_x = (bbox_coords['x1'] + bbox_coords['x2']) / 2
                    center_y = (bbox_coords['y1'] + bbox_coords['y2']) / 2
                    lat, lon = pixel_to_latlon(center_x, center_y, bounds, size)

                    feature = {
                        'type': 'Feature',
                        'geometry': {'type': 'Point', 'coordinates': [lon, lat]},
                        'properties': {
                            'confidence': det['confidence'],
                            'tile_id': result.get('image_id'),
                            'model': 'yolo',
                            'city': city_name
                        }
                    }

                    if not first:
                        f.write(',\n')
                    f.write('    ' + json.dumps(feature))
                    first = False

        f.write('\n  ]\n}\n')

    logger.info(f"{city_name} GeoJSON created: {yolo_path}")
    return yolo_path

# ============================================================================
# PART 11: RUN EVERYTHING
# ============================================================================

print("\n" + "="*70)
print("STARTING MULTI-CITY TREE DETECTION PIPELINE")
print("="*70 + "\n")

# Calculate and display tile counts for all cities
print("PRE-PROCESSING ANALYSIS")
print("-" * 70)
total_estimated_tiles = 0
for city_config in CITIES:
    tiles_coords = list(mercantile.tiles(*city_config["bbox"], zooms=ZOOM_LEVEL))
    num_tiles = len(tiles_coords)
    total_estimated_tiles += num_tiles
    print(f"  {city_config['name']:20s} : {num_tiles:6d} tiles")
print("-" * 70)
print(f"  {'TOTAL':20s} : {total_estimated_tiles:6d} tiles")
print(f"\nEstimated processing time (at ~{DOWNLOAD_THREADS} tiles/sec): {total_estimated_tiles/DOWNLOAD_THREADS/60:.1f} minutes")
print("="*70 + "\n")

# Load YOLO model ONCE at the beginning
print(f"Loading YOLO model from {YOLO_MODEL_PATH}...")
from ultralytics import YOLO
yolo_model = YOLO(YOLO_MODEL_PATH)
yolo_model.to(device)
print(f"YOLO model loaded successfully\n")

# Initialize downloader with parallel threads
downloader = TileDownloader(MAPTILER_API_KEY, num_threads=DOWNLOAD_THREADS)
downloader.start()

# Process each city
all_cities_results = {}

try:
    for city_config in CITIES:
        city_name = city_config["name"]

        # Run pipeline for this city
        results, zoom = run_pipeline_parallel(
            city_config=city_config,
            yolo_model=yolo_model,
            yolo_conf=YOLO_CONFIDENCE,
            downloader=downloader,
            device=device,
            max_tiles=MAX_TILES,
            zoom=ZOOM_LEVEL
        )

        # Store results
        all_cities_results[city_name] = {
            "results": results,
            "zoom": zoom,
            "output_folder": os.path.join(OUTPUT_BASE_FOLDER, f"{city_name}_zoom{zoom}")
        }

        # Create GeoJSON for this city
        if len(results) > 0:
            output_folder = all_cities_results[city_name]["output_folder"]
            geojson_path = create_geojson(results, output_folder, city_name)
            all_cities_results[city_name]["geojson"] = geojson_path

finally:
    # Stop downloader threads
    downloader.stop()

# ============================================================================
# PART 12: FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("MULTI-CITY DETECTION SUMMARY")
print("="*70)

total_trees = 0
total_tiles = 0
total_failed = 0

# Header for table
print(f"\n{'CITY':<20} {'TILES':>8} {'FAILED':>8} {'TREES':>10} {'AVG TREES/TILE':>15}")
print("-" * 70)

for city_name, city_data in all_cities_results.items():
    results = city_data["results"]
    successful_results = [r for r in results if "error" not in r]
    failed_results = len(results) - len(successful_results)
    yolo_detections = sum(r.get("trees_detected", 0) for r in successful_results)
    avg_trees = yolo_detections / len(successful_results) if len(successful_results) > 0 else 0

    total_trees += yolo_detections
    total_tiles += len(successful_results)
    total_failed += failed_results

    print(f"{city_name:<20} {len(successful_results):>8} {failed_results:>8} {yolo_detections:>10} {avg_trees:>15.2f}")

print("-" * 70)
print(f"{'TOTAL':<20} {total_tiles:>8} {total_failed:>8} {total_trees:>10} {total_trees/total_tiles if total_tiles > 0 else 0:>15.2f}")
print("=" * 70)

print(f"\nRESULTS LOCATION:")
print(f"   Base folder: {OUTPUT_BASE_FOLDER}")
print(f"\nOUTPUT FILES PER CITY:")
print(f"   - [city]_zoom{ZOOM_LEVEL}/")
print(f"     ├── [city]_yolo_detections.geojson  (map visualization)")
print(f"     ├── [city]_processing.log            (detailed logs)")
print(f"     └── json/tile_*_results.json         (individual tile results)")

print(f"\nPROCESSING CONFIGURATION:")
print(f"   Model: YOLO11")
print(f"   Confidence: {YOLO_CONFIDENCE}")
print(f"   Zoom level: {ZOOM_LEVEL}")
print(f"   Download threads: {DOWNLOAD_THREADS}")
print(f"   Inference batch size: {INFERENCE_BATCH_SIZE}")
print(f"   Device: {device}")

print(f"\nAll results saved to Google Drive!")
print(f"PIPELINE COMPLETE!")
print("="*70)